In [3]:
import numpy as np
import re
from pprint import pp

sample = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

def get_array(data):
    width = len(data.splitlines()[0])
    digits = re.findall(r'\d', data)
    array = np.array(digits, dtype=int)
    array.shape = len(digits) // width, width
    return array


In [4]:
def get_neighbors(shape):
    results = {}
    for y in range(shape[0]):
        for x in range(shape[1]):
            neighbors = []
            if y > 0:
                neighbors.append((y - 1,x))
            if y < shape[0] - 1:
                neighbors.append((y + 1,x))
            if x > 0:
                neighbors.append((y,x - 1))
            if x < shape[1] - 1:
                neighbors.append((y,x + 1))
            results[(y,x)] = tuple(neighbors)
    return results

class Distancer:
    def __init__(self):
        self.max_y = self.array.shape[0] - 1
        self.max_x = self.array.shape[1] - 1
    def get_dist(self, y, x):
        return self.max_y - y + self.max_x - x

class DistancerFromData(Distancer):
    def __init__(self, data):
        self.array = get_array(data)
        super().__init__()

class DistancerFromArray(Distancer):
    def __init__(self, array):
        self.array = array
        super().__init__()

In [5]:
def plot_risk(array, knowns):
    plot = np.zeros(array.shape, dtype=int)
    plot = plot - 1
    for known, risk in knowns.items():
        try:
            plot[known] = risk
        except:
            pass
    return plot

def visualize_risk(array, knowns, corner=False):
    plot = plot_risk(array, knowns)
    if corner:
        plot = plot[-10:, -10:]
    for row in plot:
        print (''.join(f'{i:x}'[-1] if i > -1 else 'X' for i in row))
    print (" --- ")




In [6]:

from operator import itemgetter
from queue import PriorityQueue
from collections import defaultdict
from sys import maxsize
third = itemgetter(2)

class NewPathFinder:
    def __init__(self, data, debug=False, limit=None):
        self.limit = limit
        self.debug = debug
        if isinstance(data,str):
            self.array = get_array(data)
        else:
            self.array = data
        self.neighbors = get_neighbors(self.array.shape)
        self.max_y = self.array.shape[0] - 1
        self.max_x = self.array.shape[1] - 1
        self.end = self.max_y, self.max_x
        self.knowns = defaultdict(lambda: maxsize)
        self.knowns[self.end] = 0
        self.todo = PriorityQueue()
        self.todo.put((0, 0, self.end))
        self.been_there = set()
        self.count = 0
    
    def get_dist(self, y, x):
        return self.max_y - y + self.max_x - x

    def get_todo(self):
        while True:
            if self.limit and self.count > self.limit:
                print ("Limit reached")
                break
            # print(f'Working: {self.todo.queue}')
            if not self.todo.empty():
                ondeck = self.todo.get()
                yield third(ondeck)
            else:
                break

    def do_stuff(self):
        for todo in self.get_todo():
            if todo in self.been_there:
                continue
            self.count+=1
            self.been_there.add(todo)
            risk_to_me = self.array[todo]
            my_risk = self.knowns[todo]
            self.debug and print(f'Regarding {todo}.  Known risk is {my_risk}')
            neighbors = self.neighbors[todo]
            for neighbor in neighbors:
                known = self.knowns[neighbor]
                new_risk = risk_to_me + my_risk
                if new_risk < known:
                    self.debug and print (f"IMPROVED! {neighbor} {known} with {new_risk}")
                    self.knowns[neighbor] = new_risk
                    dist = self.get_dist(*neighbor)
                    self.todo.put((new_risk, dist, neighbor))
                else: 
                    self.debug and print (f"ALREADY GOOD! {neighbor} {known} is better than {new_risk}")
            yield


pf = NewPathFinder(sample, limit=200, debug=False)
print(pf.array)
list(pf.do_stuff())

visualize_risk(pf.array, pf.knowns)
print(pf.knowns[0,0])
print(pf.count)

[[1 1 6 3 7 5 1 7 4 2]
 [1 3 8 1 3 7 3 6 7 2]
 [2 1 3 6 5 1 1 3 2 8]
 [3 6 9 4 9 3 1 5 6 9]
 [7 4 6 3 4 1 7 1 1 1]
 [1 3 1 9 1 2 8 1 3 7]
 [1 3 5 9 9 1 2 4 2 1]
 [3 1 2 5 4 2 1 6 3 9]
 [1 2 9 3 1 3 8 5 2 1]
 [2 3 1 1 9 4 4 5 8 1]]
882fe989ce
75feb6565c
541b654335
74ea633ede
10a732edcd
0dc320dc9a
fcb90fd979
cb940ed742
ec74309421
c98742e910
 --- 
40
100


In [7]:

pf = NewPathFinder(sample, limit=8, debug=True)
print(pf.array)
from pprint import pprint
pprint(pf.knowns)
list(pf.do_stuff())

visualize_risk(pf.array, pf.knowns)
print(pf.count)

[[1 1 6 3 7 5 1 7 4 2]
 [1 3 8 1 3 7 3 6 7 2]
 [2 1 3 6 5 1 1 3 2 8]
 [3 6 9 4 9 3 1 5 6 9]
 [7 4 6 3 4 1 7 1 1 1]
 [1 3 1 9 1 2 8 1 3 7]
 [1 3 5 9 9 1 2 4 2 1]
 [3 1 2 5 4 2 1 6 3 9]
 [1 2 9 3 1 3 8 5 2 1]
 [2 3 1 1 9 4 4 5 8 1]]
defaultdict(<function NewPathFinder.__init__.<locals>.<lambda> at 0x7f79981ac5e0>,
            {(9, 9): 0})
Regarding (9, 9).  Known risk is 0
IMPROVED! (8, 9) 9223372036854775807 with 1
IMPROVED! (9, 8) 9223372036854775807 with 1
Regarding (8, 9).  Known risk is 1
IMPROVED! (7, 9) 9223372036854775807 with 2
ALREADY GOOD! (9, 9) 0 is better than 2
IMPROVED! (8, 8) 9223372036854775807 with 2
Regarding (9, 8).  Known risk is 1
ALREADY GOOD! (8, 8) 2 is better than 9
IMPROVED! (9, 7) 9223372036854775807 with 9
ALREADY GOOD! (9, 9) 0 is better than 9
Regarding (7, 9).  Known risk is 2
IMPROVED! (6, 9) 9223372036854775807 with 11
ALREADY GOOD! (8, 9) 1 is better than 11
IMPROVED! (7, 8) 9223372036854775807 with 11
Regarding (8, 8).  Known risk is 2
IMPROVED! (7, 8

In [8]:
realdeal = open('d15.input').read()
array = get_array(realdeal)
pf = NewPathFinder(array, limit=99999, debug=False)
list(pf.do_stuff())
print(pf.count)
print(pf.knowns.get((0,0)))

10000
790


In [9]:
sample_embiggened = """11637517422274862853338597396444961841755517295286
13813736722492484783351359589446246169155735727126
21365113283247622439435873354154698446526571955763
36949315694715142671582625378269373648937148475914
74634171118574528222968563933317967414442817852555
13191281372421239248353234135946434524615754563572
13599124212461123532357223464346833457545794456865
31254216394236532741534764385264587549637569865174
12931385212314249632342535174345364628545647573965
23119445813422155692453326671356443778246755488935
22748628533385973964449618417555172952866628316397
24924847833513595894462461691557357271266846838237
32476224394358733541546984465265719557637682166874
47151426715826253782693736489371484759148259586125
85745282229685639333179674144428178525553928963666
24212392483532341359464345246157545635726865674683
24611235323572234643468334575457944568656815567976
42365327415347643852645875496375698651748671976285
23142496323425351743453646285456475739656758684176
34221556924533266713564437782467554889357866599146
33859739644496184175551729528666283163977739427418
35135958944624616915573572712668468382377957949348
43587335415469844652657195576376821668748793277985
58262537826937364893714847591482595861259361697236
96856393331796741444281785255539289636664139174777
35323413594643452461575456357268656746837976785794
35722346434683345754579445686568155679767926678187
53476438526458754963756986517486719762859782187396
34253517434536462854564757396567586841767869795287
45332667135644377824675548893578665991468977611257
44961841755517295286662831639777394274188841538529
46246169155735727126684683823779579493488168151459
54698446526571955763768216687487932779859814388196
69373648937148475914825958612593616972361472718347
17967414442817852555392896366641391747775241285888
46434524615754563572686567468379767857948187896815
46833457545794456865681556797679266781878137789298
64587549637569865174867197628597821873961893298417
45364628545647573965675868417678697952878971816398
56443778246755488935786659914689776112579188722368
55172952866628316397773942741888415385299952649631
57357271266846838237795794934881681514599279262561
65719557637682166874879327798598143881961925499217
71484759148259586125936169723614727183472583829458
28178525553928963666413917477752412858886352396999
57545635726865674683797678579481878968159298917926
57944568656815567976792667818781377892989248891319
75698651748671976285978218739618932984172914319528
56475739656758684176786979528789718163989182927419
67554889357866599146897761125791887223681299833479"""

In [10]:
array = get_array(sample_embiggened)
pf = NewPathFinder(array, limit=99999, debug=False)
list(pf.do_stuff())
print(pf.count)
print(pf.knowns.get((0,0)))

2500
315


In [11]:
def embiggen(array):
    ogwidth = array.shape[1]
    ogheight = array.shape[0]
    for i in range(4):
        start = ogheight * i
        stop = start + ogwidth
        bumped = array[start:stop,:] + 1
        rolls = bumped > 9
        rolled = np.where(rolls, 1, bumped)
        array = np.vstack((array, rolled))
    for i in range(4):
        start = ogwidth * i
        stop = start + ogwidth
        bumped = array[:,start:stop] + 1
        rolls = bumped > 9
        rolled = np.where(rolls, 1, bumped)
        array = np.hstack((array,rolled))
    return array




(embiggen(get_array(sample)) == get_array(sample_embiggened)).all()

True

In [12]:
realdeal = open('d15.input').read()
array = get_array(realdeal)
print(array.shape)
embiggened_real_deal = embiggen(array)
print(embiggened_real_deal.shape)
pf = NewPathFinder(embiggened_real_deal, limit=999999, debug=False)
list(pf.do_stuff())
print(pf.count)
print(pf.knowns.get((0,0)))


(100, 100)
(500, 500)
250000
2998
